# Hodiny 18–20 — Projekt: Závěrečný mini‑projekt

Tento tříhodinový blok je věnován projektu: žáci použijí nástroje a algoritmy, které se naučili (hledání cest, heuristiky, backtracking, constraint solving), aby vytvořili vlastní řešení zvoleného problému. Níže jsou tři doporučené projektové směry a šablony, které mohou studenti použít jako výchozí bod.

## Co je cílem?

- Vytvořit funkční program, který řeší vybraný problém (N‑Queens, Sudoku, Hanojská věž, nebo cesta v bludišti).
- Implementovat a zhodnotit minimálně dva přístupy (např. backtracking vs. constraint solver, nebo BFS vs. A*).
- Připravit krátkou prezentaci (5–10 minut): popis problému, ukázka kódu, metriky (délka řešení / čas / počet rozšířených stavů) a vysvětlení volby algoritmu.

### Doporučené projekty (šablony):

1) N‑Queens (řešení pomocí backtrackingu a pomocí `ortools` nebo `python-constraint`).

2) Sudoku (backtracking vs. CP‑solver — `ortools.sat.python.cp_model`).

3) Hanojská věž (rekurzivní řešení + stavový prostor, experiment s vizualizací kroků).

4) Cesta v bludišti (rozšíření sešitu z hodin 16–17): implementujte generátor, různá hledání (BFS, A*, Dijkstra) a porovnejte metriky.



In [3]:
# N-Queens template (backtracking) — students can use this as starting point

def solve_n_queens(n):
    solutions = []
    cols = set()
    diag1 = set()
    diag2 = set()
    board = [-1]*n

    def place(row):
        if row == n:
            solutions.append(board.copy())
            return
        for c in range(n):
            if c in cols or (row-c) in diag1 or (row+c) in diag2:
                continue
            cols.add(c); diag1.add(row-c); diag2.add(row+c)
            board[row] = c
            place(row+1)
            cols.remove(c); diag1.remove(row-c); diag2.remove(row+c)
            board[row] = -1

    place(0)
    return solutions

# quick test
sol4 = solve_n_queens(4)
print('Solutions for 9-queens:', len(sol4))
assert len(sol4) == 2
print('N-Queens test passed')

Solutions for 9-queens: 2
N-Queens test passed


In [10]:
# Sudoku using OR-Tools CP-SAT (simple 4x4 example for speed in classroom)
try:
    from ortools.sat.python import cp_model
except Exception as e:
    cp_model = None
    print('ortools not installed; students can install with pip install ortools')


def solve_sudoku(grid):
    if not cp_model:
        return None
    n = len(grid)
    model = cp_model.CpModel()
    cells = [[model.NewIntVar(1, n, f'x{i}{j}') for j in range(n)] for i in range(n)]
    for i in range(n):
        model.AddAllDifferent(cells[i])
    for j in range(n):
        model.AddAllDifferent([cells[i][j] for i in range(n)])
    # Add block constraints for sqrt(n) x sqrt(n) blocks if n is a perfect square
    import math
    sqrt_n = int(math.sqrt(n))
    if sqrt_n * sqrt_n == n:
        for bi in range(sqrt_n):
            for bj in range(sqrt_n):
                block = [cells[bi * sqrt_n + ii][bj * sqrt_n + jj] for ii in range(sqrt_n) for jj in range(sqrt_n)]
                model.AddAllDifferent(block)
    # fill known
    for i in range(n):
        for j in range(n):
            if grid[i][j] != 0:
                model.Add(cells[i][j] == grid[i][j])
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return [[int(solver.Value(cells[i][j])) for j in range(n)] for i in range(n)]
    return None

# tiny test (4x4)
example = [[1,0,0,0],[0,0,2,0],[0,3,0,0],[0,0,0,4]]
# Example 9x9 Sudoku puzzle (0 means empty)
example_9x9 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]
res = solve_sudoku(example_9x9)
if res:
    print('Sudoku 9x9 solved sample:')
    for row in res:
        print(row)
else:
    print('Sudoku solver not available or no solution')

Sudoku 9x9 solved sample:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


## Prezentace a hodnocení

Každý tým/student připraví:
- 5–10 minutovou prezentaci: problém, použité algoritmy, ukázka kódu (nejdůležitější funkce), výsledky (metriky: čas, délka, rozšířené uzly).
- Krátké demo (spustitelný skript nebo notebook cell) ukazující řešení.

Hodnocení (doporučení):
- Funkčnost (40 %) — řešení pracuje pro zadané případy.
- Čistota kódu a modularita (20 %).
- Porovnání přístupů a měření výkonu (20 %).
- Prezentace a vysvětlení rozhodnutí (20 %).

### Doporučené výstupy
- `solution.py` nebo `notebook` s hlavním runnerem.
- Stručný `README.md` s instrukcemi jak spustit.
- Soubor `results.csv` s metrikami z benchmarků (volitelné).